# 1. mp3 -> wav

In [10]:
import os, tqdm, shutil

subset = "train"
# subset = "valid"
# subset = "test.2016"
# subset = "test.2017"
# subset = "test.coco"

mp3_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/es_mp3"
wav_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/es"
os.makedirs(os.path.join(wav_root, subset), exist_ok=True)
for file in tqdm.tqdm(os.listdir(os.path.join(mp3_root, subset))):
    if file.endswith(".json"):
        shutil.copy(
            os.path.join(mp3_root, subset, file), os.path.join(wav_root, subset, file)
        )
    elif file.endswith(".mp3"):
        cmd = f"ffmpeg -i {os.path.join(mp3_root, subset, file)} -ar 16000 {os.path.join(wav_root, subset, file[:-4] + '.wav')} -loglevel quiet"
        os.system(cmd)

100%|██████████| 58000/58000 [54:19<00:00, 17.79it/s]  


In [11]:
import os, glob
from pydub import AudioSegment
subset = "train"
wav_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/16khz_wav/es"
wavs = glob.glob(f"{os.path.join(wav_root, subset)}/*.wav")
print(len(wavs))
wav = wavs[0]
audio = AudioSegment.from_wav(wav)
print(audio.frame_rate)

29000
16000


# 2. manifest

# 3. Quantize using the learned clusters

# 4. postprocess quatized txt

In [19]:
import os
from pathlib import Path
from itertools import groupby

# subset = "train"
# subset = "valid"
# subset = "test.2016"
# subset = "test.2017"
subset = "test.coco"

lang = "es"
manifest_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/manifest"
manifest_dir = Path(manifest_root) / f"{subset}.{lang}"

manifest_lines = []
quantized_lines = []
with open(manifest_dir / f"{subset}.{lang}.tsv", mode="r+") as f:
    lines = f.readlines()
    for line in lines[1: ]:
        manifest_lines.append(line.strip())
with open(manifest_dir / f"{subset}.{lang}.txt", mode="r+") as f:
    for i, line in enumerate(f.readlines()):
        line = line.strip()
        index, unit = line.split('|')
        assert index == manifest_lines[i].split('\t')[0].split('.')[0]
        unit = unit.split()
        unit = [k for k, _ in groupby(unit)]
        unit = ' '.join([str(k) for k in unit])
        quantized_lines.append(index + "|" + unit)
        # print(quantized_lines[-1])
with open(manifest_dir / f"{subset}.{lang}.txt", mode="w+") as f:
    f.write('\n'.join(quantized_lines))

# 6. generate es-en tsv from es and en source_unit tsv

In [ ]:
import os
import pandas as pd
from pathlib import Path
from itertools import groupby

# src_lang, tgt_lang = "es", "en"
# src_lang, tgt_lang = "en", "fr"
src_lang, tgt_lang = "en", "es"

# subset = "train"
# subset = "valid"
# subset = "test.2016"
# subset = "test.2017"
subset = "test.coco"

output_manifest_root = f"/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/format_data/{src_lang}-{tgt_lang}"

src_manifest_root = f"/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/format_data/{src_lang}-{tgt_lang}/{src_lang}_source_unit"
src_manifest_path = Path(src_manifest_root) / f"{subset}.tsv"

tgt_manifest_root = "/opt/data/private/dsy/project/dataset/multi30k-dataset/data/speech/manifest"
tgt_manifest_dir = Path(tgt_manifest_root) / f"{subset}.{tgt_lang}"

src_tsv = pd.read_csv(src_manifest_path, sep='\t')
id2tgt_unit = {}
tgt_unit = []
tgt_size = []
with open(tgt_manifest_dir / f"{subset}.{tgt_lang}.txt", mode="r+") as f:
    for line in f.readlines():
        line = line.strip()
        index, unit = line.split('|')
        id2tgt_unit[int(index)] = unit
for i, row in src_tsv.iterrows():
    unit = id2tgt_unit[row["id"]]
    tgt_unit.append(unit)
    tgt_size.append(unit.count(' ') + 1)
assert len(tgt_unit) == len(tgt_size) == src_tsv.shape[0]
src_tsv["tgt_text"] = tgt_unit
src_tsv["tgt_n_frames"] = tgt_size

src_tsv.set_index("id", inplace=True)
src_tsv.to_csv(Path(output_manifest_root) / f"{subset}.tsv", sep='\t')